In [42]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [43]:
from convnext import *

# Thư viện PyTorch cho việc xây dựng và huấn luyện mạng nơ-ron
import torch.nn as nn  # Các module, class và hàm tính toán trong PyTorch
import torch.optim as optim  # Thuật toán tối ưu hóa cho việc cập nhật trọng số của mạng
from torch.utils.data import DataLoader  # Cung cấp DataLoader để tải dữ liệu
import torch  # Thư viện cơ bản của PyTorch, chứa các cấu trúc dữ liệu và hàm tính toán trên tensor

# Thư viện torchvision cho xử lý ảnh
from torchvision import transforms  # Các phép biến đổi dữ liệu ảnh
from torchvision.datasets import ImageFolder  # Dataset từ các thư mục chứa ảnh

# Thư viện hỗ trợ tương tác với hệ thống file và thư mục
import os


In [44]:
# Kiểm tra xem GPU có sẵn không
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [45]:
# Đường dẫn tới thư mục chứa dữ liệu
base_dir = '../data'

In [46]:
# Định nghĩa các phép biến đổi dữ liệu
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [47]:
# Tạo dataset cho dữ liệu huấn luyện từ thư mục train_data
train_data = ImageFolder(root=os.path.join(base_dir, 'train'), transform=transform)

# Tạo dataset cho dữ liệu validation từ thư mục valid_data
valid_data = ImageFolder(root=os.path.join(base_dir, 'valid'), transform=transform)

In [48]:
# Sử dụng hàm Lambda để gán nhãn
train_data.class_to_idx = {'no-watermark': 0, 'watermark': 1}
train_data.targets = [label for _, label in train_data.samples]

valid_data.class_to_idx = {'no-watermark': 0, 'watermark': 1}
valid_data.targets = [label for _, label in valid_data.samples]

In [49]:
batch_size = 50
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)

In [ ]:
def train_model(train_loader, valid_loader, model, num_epochs):
    best_valid_accuracy = 0.0  # Khởi tạo biến lưu trữ valid_accuracy cao nhất

    num_epochs = num_epochs

    # Lặp qua từng epoch
    for epoch in range(num_epochs):
        # Khởi tạo các biến đếm
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        # Lặp qua từng batch dữ liệu huấn luyện
        for batch_data, batch_labels in train_loader:
            # Chuyển dữ liệu và nhãn lên GPU
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)

            # Xóa gradient của các tham số
            optimizer.zero_grad()

            # Đưa dữ liệu vào mô hình để tính toán output
            outputs = model(batch_data)

            # Tính toán giá trị loss
            loss = criterion(outputs, batch_labels)

            # Lan truyền ngược để tính gradient
            loss.backward()

            # Cập nhật trọng số của mô hình
            optimizer.step()

            # Tính toán training accuracy
            _, predicted_train = torch.max(outputs, 1)
            total_train += batch_labels.size(0)
            correct_train += (predicted_train == batch_labels).sum().item()

            # Cập nhật running loss
            running_loss += loss.item()

        # Đánh giá mô hình trên dữ liệu validation sau mỗi epoch
        with torch.no_grad():
            model.eval()
            val_loss = 0.0
            correct_val = 0
            total_val = 0
            for inputs_val, labels_val in valid_loader:
                inputs_val, labels_val = inputs_val.to(device), labels_val.to(device)
                outputs_val = model(inputs_val)
                val_loss += criterion(outputs_val, labels_val).item()
                _, predicted_val = torch.max(outputs_val, 1)
                total_val += labels_val.size(0)
                correct_val += (predicted_val == labels_val).sum().item()

            # Tính toán valid_accuracy
            valid_accuracy = 100 * correct_val / total_val

            # Kiểm tra và lưu mô hình nếu valid_accuracy cao hơn
            if valid_accuracy > best_valid_accuracy:
                best_valid_accuracy = valid_accuracy
                torch.save(model, 'improved_watermark_detection_model.pth')

        # In ra kết quả sau mỗi epoch
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Training Accuracy: {(100 * correct_train / total_train)}%, Validation Loss: {val_loss/len(valid_loader)}, Validation Accuracy: {valid_accuracy}%")

        # Chuyển lại sang chế độ huấn luyện
        model.train()
    
    print(f"best valid accuracy: {best_valid_accuracy}")

In [50]:
# Khởi tạo mô hình ConvNeXt
model = convnext_tiny(pretrained=False, in_22k=False, num_classes = 2).to(device)

# Chuyển mô hình vào chế độ huấn luyện
model.train()

# Chọn hàm loss và trình tối ưu hóa
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [51]:
# Khởi tạo mô hình cải tiến
model = convnext_tiny(pretrained=False, in_22k=False, num_classes=2).to(device)
model.head = nn.Sequential( 
            nn.Linear(in_features=768, out_features=448),
            nn.Dropout(0.25),
            nn.Linear(in_features=448, out_features=2),
        ).to(device)
# Chuyển mô hình vào chế độ huấn luyện
model.train()

# Chọn hàm loss và trình tối ưu hóa
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
train_model(train_loader, valid_loader, model, 30)